In [1]:
%%time

import pandas as pd
import numpy as np
from scipy import sparse
import tarfile as tar

CPU times: user 202 ms, sys: 40 ms, total: 242 ms
Wall time: 281 ms


In [2]:
%%time

with tar.open("Webscope_C15.tgz", "r") as tar_ref:
    tar_ref.extractall()

CPU times: user 6.92 s, sys: 1.74 s, total: 8.66 s
Wall time: 16.3 s


In [3]:
%%time

with tar.open("ydata-ymusic-kddcup-2011-track1.tar.bz2", "r") as tar_ref:
    tar_ref.extractall()

CPU times: user 3min 30s, sys: 5.56 s, total: 3min 36s
Wall time: 3min 39s


In [4]:
%%time

fileIN = open("ydata-ymusic-kddcup-2011-track1/validationIdx1.txt", "r+")
fileOUT = open("test_tmp.txt", "w+")


lineCount = 0
for line in fileIN:
    if lineCount == 0:
        token = line.strip().split('|')
        uid = token[0]
        lineCount = int(token[1])
    else:
        token = line.strip().split()
        iid = token[0]
        score = token[1]
        lineCount = lineCount - 1;
        fileOUT.write(str(uid) + "\t" + str(iid) + "\t" + str(score) + "\n")

fileIN.close()
fileOUT.close()

CPU times: user 8.33 s, sys: 91.8 ms, total: 8.42 s
Wall time: 8.63 s


In [5]:
%%time

fileIN = open("ydata-ymusic-kddcup-2011-track1/trainIdx1.txt", "r+")
fileOUT = open("train_tmp.txt", "w+")

lineCount = 0
for line in fileIN:
    if lineCount == 0:
        token = line.strip().split('|')
        uid = token[0]
        lineCount = int(token[1])
    else:
        token = line.strip().split()
        iid = token[0]
        score = token[1]
        lineCount = lineCount - 1;
        fileOUT.write(str(uid) + "\t" + str(iid) + "\t" + str(score) + "\n")

fileIN.close()
fileOUT.close()

CPU times: user 8min 3s, sys: 5.67 s, total: 8min 8s
Wall time: 8min 9s


In [6]:
m = 1_000_990
n = 624_961

nnz_train = 252_800_275
nnz_test = 4_003_960

train_data_file = "train_tmp.txt"
test_data_file = "test_tmp.txt"

print("preparing training data")
%time test_data = pd.read_csv(test_data_file, delimiter = "\t",header=None)
test_user = test_data[0].values
test_item = test_data[1].values
test_rating = test_data[2].values

print("preparing test data")
%time train_data = pd.read_csv(train_data_file, delimiter = "\t",header=None)
train_user = train_data[0].values
train_item = train_data[1].values
train_rating = train_data[2].values

preparing training data
CPU times: user 1.3 s, sys: 347 ms, total: 1.65 s
Wall time: 1 s
preparing test data
CPU times: user 1min 5s, sys: 22.4 s, total: 1min 27s
Wall time: 54.8 s


In [7]:
%%time

print(test_user)
print(test_item)
print(test_rating)
print("")
print(np.max(test_user))
print(np.max(test_item))
print(np.max(test_rating))
print("")
print(np.min(test_user))
print(np.min(test_item))
print(np.min(test_rating))
print("")
print(np.unique(test_user).size)
print(np.unique(test_item).size)
print(np.unique(test_rating).size)
print("")
print(test_user.size)

assert test_user.size == nnz_test

[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]
[70 90 70 ... 90 90 90]

1000989
624960
100

0
2
0

1000990
258694
101

4003960
CPU times: user 6.77 s, sys: 48.3 ms, total: 6.82 s
Wall time: 532 ms


In [8]:
%%time

print(train_user)
print(train_item)
print(train_rating)
print("")
print(np.max(train_user))
print(np.max(train_item))
print(np.max(train_rating))
print("")
print(np.min(train_user))
print(np.min(train_item))
print(np.min(train_rating))
print("")
print(np.unique(train_user).size)
print(np.unique(train_item).size)
print(np.unique(train_rating).size)
print("")
print(train_user.size)

assert train_user.size == nnz_train
assert np.max(train_user)+1 == m  # ids start with 0
assert np.max(train_item)+1 == n  # ids start with 0

[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]
[90 90 90 ... 80 90 90]

1000989
624960
100

0
0
0

1000990
624961
101

252800275
CPU times: user 42.2 s, sys: 6.66 s, total: 48.9 s
Wall time: 31.2 s


In [9]:
%%time

R_test_coo = sparse.coo_matrix((test_rating, (test_user, test_item)))
assert R_test_coo.nnz == nnz_test

# 0-based to 1-based
outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str(test_user[i] + 1) + " " + str(test_item[i] + 1) + " " + str(test_rating[i]) + "\n")

CPU times: user 13.2 s, sys: 88.3 ms, total: 13.3 s
Wall time: 13.3 s


In [10]:
%%time
# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 9.24 ms, sys: 56.3 ms, total: 65.5 ms
Wall time: 63.2 ms


In [11]:
print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[70 90 70 ... 90 90 90]
[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]

[70. 90. 70. ... 90. 90. 90.]
[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]


In [12]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(test_user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(test_item).size)
print(np.unique(R_test_coo.col).size)

100
1000989
624960

0
0
2

1000990
1000990
258694
258694
CPU times: user 9.86 s, sys: 60.1 ms, total: 9.92 s
Wall time: 779 ms


In [13]:
%%time

R_train_coo = sparse.coo_matrix((train_rating, (train_user, train_item)))
assert R_train_coo.nnz == nnz_train

# 0-based to 1-based
outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str(train_user[i] + 1) + " " + str(train_item[i] + 1) + " " + str(train_rating[i]) + "\n")

CPU times: user 14min 29s, sys: 9.26 s, total: 14min 38s
Wall time: 14min 33s


In [14]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 42.6 s, sys: 9.85 s, total: 52.4 s
Wall time: 54.7 s


In [15]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 55.1 ms, sys: 4.78 s, total: 4.83 s
Wall time: 1min 2s


In [16]:
print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[90 90 90 ... 80 90 90]
[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]

[90. 90. 90. ... 80. 90. 90.]
[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]

[90 90 70 ... 80 90 90]
[  9101  22757  26137 ... 531386 532569 556366]
[        0        41       270 ... 252800219 252800246 252800275]

[90. 90. 70. ... 80. 90. 90.]
[  9101  22757  26137 ... 531386 532569 556366]
[        0        41       270 ... 252800219 252800246 252800275]

[50 83 71 ... 90  0 90]
[  2435 101039 208370 ... 951381 951516 987296]
[        0        20        49 ... 252800191 252800246 252800275]

[50. 83. 71. ... 90.  0. 90.]
[  2435 101039 208370 ... 951381 951516 987296]
[        0        20        49 ... 252800191 252800246 252800275]


In [17]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(train_user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(train_item).size)
print(np.unique(R_train_coo.col).size)

100
1000989
624960

0
0
0

1000990
1000990
624961
624961
CPU times: user 58.8 s, sys: 11.5 s, total: 1min 10s
Wall time: 47.5 s


In [18]:
%%time
print("write extra meta file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

write extra meta file
CPU times: user 979 µs, sys: 43 µs, total: 1.02 ms
Wall time: 3.88 ms


In [19]:
%%time
print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta file
CPU times: user 622 µs, sys: 0 ns, total: 622 µs
Wall time: 1.22 ms
